In [ ]:
import datetime
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
data_original_path = 'public_data.csv'
data_predicted_path = 'predicted_data.csv'

In [ ]:
df = pd.read_csv(
    data_original_path
)

In [ ]:
df = df[df.index > 800]

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
df.head(len(df))

In [ ]:
def country_df(country_name, df):
    ds_current = df['date']
    y_current = df[country_name]
    globals()['df_' + country_name] = pd.DataFrame({
        'ds': ds_current, 'y': y_current
    })
    globals()['df_' + country_name]['ds'] = pd.to_datetime(globals()['df_' + country_name]['ds'])
    globals()['df_' + country_name]['y'] = globals()['df_' + country_name]['y'].astype('float64')
    print(f'df_{country_name} added')

In [ ]:
def show_df_info(df):
    df.plot(x='ds', y='y', figsize=(18, 6))

In [ ]:
country_list = df.columns[1:]

In [ ]:
for country_name in country_list:
    country_df(country_name, df)
    show_df_info(globals()['df_' + country_name])

In [ ]:
m = Prophet()

In [ ]:
def add_country_prophet(country_name):
    globals()['m_' + country_name] = Prophet()
    print(f'm_{country_name} Prophet added')

In [ ]:
for country_name in country_list:
    add_country_prophet(country_name)

In [ ]:
def fit_n_train_prophet(df_local, country_name):
    m = globals()['m_' + country_name]
    m.add_seasonality(name='weekly', period=7, fourier_order=3)
    m.add_seasonality(name='daily', period=1, fourier_order=3)
    m.fit(df_local)

    future = m.make_future_dataframe(periods=31)
    forecast = m.predict(future)
    # print(forecast.head(), forecast.tail(), sep='\n')
    # print('\n\n')
    return forecast

In [ ]:
country_forecasts = {name: None for name in country_list}
country_forecasts

In [ ]:
for country_name in country_list:
    current_df = globals()['df_' + country_name]
    country_forecasts[country_name] = fit_n_train_prophet(current_df, country_name)

In [ ]:
country_forecasts['Brazil']

In [ ]:
def create_graph(country_name):
    m = globals()['m_' + country_name]
    forecast = country_forecasts[country_name]
    pl = plot_plotly(m, forecast)
    pl.update_layout(title=f'{country_name} Forecast thingy')
    return pl
    

In [ ]:
country_forecasts_plots = {name: None for name in country_list}

In [ ]:
for country_name in country_list:
    country_forecasts_plots[country_name] = create_graph(country_name)

In [ ]:
country_forecasts_plots['Brazil'].show()

In [ ]:
country_forecasts_results = {name: None for name in country_list}

In [ ]:
for country_name in country_list:
    country_forecasts_results[country_name] = country_forecasts[country_name][['ds', 'yhat']]
    country_forecasts_results[country_name] = country_forecasts_results[country_name][-31:].reset_index(drop=True)
    country_forecasts_results[country_name] = country_forecasts_results[country_name].rename(columns={'yhat': 'y', 'ds': 'date'})

In [ ]:
country_forecasts_results['Brazil'].info()

In [ ]:
country_forecasts_results['Brazil'].tail()

In [ ]:
start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 5, 31)
date_range = pd.date_range(start=start_date, end=end_date)

In [ ]:
df_result = pd.DataFrame({'date': date_range})

In [ ]:
for column in country_list:
    df_result.insert(loc=0, column=column, value=None)
df_result = df_result.loc[:, ::-1]

In [ ]:
for country_name in country_list:
    for i in range(len(country_forecasts_results[country_name])):
        df_result[country_name][i] = country_forecasts_results[country_name]['y'][i]

In [ ]:
df_result

In [ ]:
country_forecasts_results['Brazil'].head()

In [ ]:
country_forecasts_plots['Brazil'].show()

In [ ]:
df_result.to_csv(data_predicted_path, index=False)